In [ ]:
# OCR
!apt install ocrmypdf
# Language pack
!apt install tesseract-ocr-rus
# pdf -> txt export tool
!apt install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript icc-profiles-free libgs9
  libgs9-common libidn12 libijs-0.35 libimagequant0 libjbig2dec0 libqpdf28 libraqm0 mailcap
  mime-support pngquant poppler-data python3-bs4 python3-chardet python3-coloredlogs
  python3-html5lib python3-humanfriendly python3-img2pdf python3-lxml python3-olefile
  python3-packaging python3-pdfminer python3-pikepdf python3-pil python3-pluggy python3-renderpm
  python3-reportlab python3-reportlab-accel python3-soupsieve python3-tqdm python3-webencodings
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd unpaper
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x ocrmypdf-doc
  python-watchdog img2pdf poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-go

In [ ]:
# !curl -o archive.zip "https://gitlab.com/alekseibogdan1/ocr_project/-/raw/main/archive.zip"
!unzip archive.zip
!rm archive.zip
%cd archive

Archive:  archive.zip
  inflating: archive/a-bashlachev-i-t-kibirov-printsipy-konstituirovaniya-liricheskogo-subekta.pdf  
  inflating: archive/abbreviatury-vchera-i-segodnya.pdf  
  inflating: archive/a-bogdanov-i-a-platonov-k-postanovke-voprosa.pdf  
  inflating: archive/administrative-reform-and-the-new-conflict-of-the-faculties-at-french-universities.pdf  
  inflating: archive/adresant-v-aspekte-yazykovoy-otsenki-na-materiale-pisem-m-i-tsvetaevoy.pdf  
  inflating: archive/a-e-kulakovskiy-voprosy-vyzhivaniya-yakutskogo-naroda-v-globalnom-mire-po-poeme-son-shamana.pdf  
  inflating: archive/afganskie-poslovitsy-i-pogovorki-v-stihotvornom-sobranii-abd-al-hamida-momanda.pdf  
  inflating: archive/aksiologicheskie-diady-v-obuchenii-inostrannyh-uchaschihsya-na-materiale-frazeologicheskih-edinits-frazeosemanticheskogo-polya.pdf  
  inflating: archive/aksiologicheskiy-potentsial-prozvisch-kak-chasti-karnavalnoy-kartiny-mira-na-materiale-britanskih-i-amerikanskih-prozvisch.pdf  
  inflatin

In [ ]:
!pwd

/content/archive


In [ ]:
!shopt -s nullglob
!for file in *.pdf; do echo $file; filename=${file%.*}; ocrmypdf $file "${filename}"_out.pdf -l rus+eng --force-ocr; pdftotext -layout "${filename}"_out.pdf "${filename}"_out.txt; done

a-bashlachev-i-t-kibirov-printsipy-konstituirovaniya-liricheskogo-subekta.pdf
Scanning contents: 100% 3/3 [00:00<00:00, 122.97page/s]
Start processing 2 pages concurrently
    2 page already has text! - rasterizing text and running OCR anyway
    1 page already has text! - rasterizing text and running OCR anyway
    3 page already has text! - rasterizing text and running OCR anyway
OCR: 100% 3.0/3.0 [00:40<00:00, 13.43s/page]
Postprocessing...
PDF/A conversion: 100% 3/3 [00:05<00:00,  1.70s/page]
Some input metadata could not be copied because it is not permitted in PDF/A. You may wish to examine the output PDF's XMP metadata.
Recompressing JPEGs: 0image [00:00, ?image/s]
Deflating JPEGs: 0image [00:00, ?image/s]
JBIG2: 0item [00:00, ?item/s]
Optimize ratio: 1.00 savings: -0.1%
Image optimization did not improve the file - optimizations will not be used
Output file is a PDF/A-2B (as expected)
The output file size is 3.76× larger than the input file.
Possible reasons for this include:
T

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
  device = torch.device('cpu')
device

device(type='cpu')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast
MODEL_NAME = 'UrukHan/t5-russian-spell'
MAX_INPUT = 256
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
from pathlib import Path
def pipeline(file_in: Path, file_out: Path):
  with open(file_in, 'r', encoding='UTF-8') as f:
    lines = f.read()
    lines = lines.replace('\n', ' ')
    chunks, chunk_size = len(lines), len(lines)//256
    input_sequences = [ lines[i:i+chunk_size] for i in range(0, chunks, chunk_size) ]

  task_prefix = "Spell correct: "
  if type(input_sequences) != list: input_sequences = [input_sequences]
  encoded = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=MAX_INPUT,
    truncation=True,
    return_tensors="pt",
  ).to(device)

  model.to(device)
  predicts = model.generate(**encoded.to(device))
  result = ' '.join(tokenizer.batch_decode(predicts, skip_special_tokens=True))

  with open(file_out, 'w', encoding='UTF-8') as res:
    res.write(result)

  del input_seqeunces
  del predicts
  del result
  del encoded

In [ ]:
Path("./fixed/").mkdir(exist_ok=True)
for filename in Path(".").glob("*.txt"):
  pipeline(filename,
           filename.parent / "fixed" / filename.name)
# pipeline(Path('/content/neologizmy-semanticheskoy-gruppy-vneshnost-cheloveka_out.txt'),
#          Path('/content/fixed/neologizmy-semanticheskoy-gruppy-vneshnost-cheloveka_out.txt'))

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
! zip fixed_texts.zip -r fixed

In [ ]:
from google.colab import files
files.download('fixed_texts.zip')